# Ray Tracer

## Einleitung

Auf Deutsch bezeichnet der englische Begriff „Ray tracing“ das „Nachverfolgen von Strahlen“. Genauer gesagt: Lichtstrahlen. Als Grafik-Technologie ermöglicht Ray tracing die Simulierung von Lichteffekten. Ein Algorithmus sendet hierzu Strahlen in einer 3D-Grafik aus, verfolgt die Strahlenpfade und berechnet ein realistisches Beleuchtungsmodell.

In der realen Welt senden Lichtquellen Lichtstrahlen aus, die dann auf Objekte treffen, dort gebrochen, reflektiert oder gestreut werden und schlussendlich bei der Kamera ankommen.
Die von der Kamera erfassten Lichtstrahlen sind deshalb nur ein Bruchteil der Lichtstrahlen, die von der Lichtquelle zwar ausgesendet, von der Kamera aber nicht aufgenommen werden.

Um nur die, für die Kamera wichtigen, Lichtstrahlen zu berechnen, wird der Weg des Lichts beim Ray tracing entgegen der natürlichen Richtung berechnet.
Die Lichtstrahlen werden also von der Kamera ausgesendet und zurückverfolgt. Die folgende Grafik zeigt wie ein Ray aus der Kamera durch die Bildebene in den drei dimensionalen Raum gesendet wird.

<img src="assets/rayTracingOverview.png" width=500px></img>

Beim Ray tracing werden ausgehend vom Augpunkt für jeden Pixel der Bildebene wird ein Ray ausgesendet. Die Rays durchstoßen die Bildebene und treffen dann auf die Objekte im drei dimensionalen Raum.
Die Farbe eines jeden Pixels der Bildebene wird durch die sog. Rendergleichung berechnet.

## Was ist ein Ray

Ein Ray, oder auch ein Lichtstrahl, ist die Kombination aus einem Ursprung und einer Richtung.
Ursprung und Richtung eines Rays können jeweils durch einen Vektor angegeben werden. Durch die Linearkombination 
des Ortsvektors (Ursprung) und des Richtungsvektors kann so jeder beliebige Punkt auf einem Ray angegeben werden.

<img src="assets/ray.jpg" width=500px></img>

Die Gleichung für eine Ray ist also folgenden: $\vec{Ray(t)}=\vec{A}+\vec{B}*t$

Die einfachste Implementierung eins Rays wäre also folgende:

In [1]:
class Ray:
  """Info of the ray and the media it's travelling"""

  def __init__(self, origin, dir):
      self.origin = origin  # the point where the ray comes from
      self.dir = dir  # direction of the ray